#### This notebook demonstrates the use of Reweighing pre-processing, Adversarial Debiasing in-processing and Reject Option Classification (ROC) post-processing algorithms for bias mitigation.
- Load imports
- Dataset
    * Load Adult, COMPAS, or German dataset and set privileged and unprivileged groups
    * Divide the dataset into training, validation, and testing partitions
    * Show dataset properties
- Pre-processing: Reweighing.
    * Show difference in mean outcomes for original training data
    * Assign weights with reweighing
    * Show difference in mean outcomes for transformed training data
- In-processing: Adversarial Debiasing.
    * Train model without debiasing, predict, and show metrics
    * Train model with debiasing, predict, and show metrics
- Post-processing: Reject Option Classification (ROC).
    * Show metrics for test set from Adversarial Debiasing without debiasing
    * Fit ROC model
    * Transform labels and show metrics

In [1]:
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
import tensorflow as tf
from warnings import warn 

# Avoid deprecation warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas

from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification

from common_utils import compute_metrics

from IPython.display import Markdown, display
from ipywidgets import interactive, FloatSlider

#### Load dataset and specify options

In [2]:
## import dataset
dataset_used = "german" # "adult", "german", "compas"
protected_attribute_used = 1 # 1, 2

if dataset_used == "adult":
#     dataset_orig = AdultDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        dataset_orig = load_preproc_data_adult(['sex'])
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
        dataset_orig = load_preproc_data_adult(['race'])
    
elif dataset_used == "german":
#     dataset_orig = GermanDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
        dataset_orig = load_preproc_data_german(['sex'])
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
        dataset_orig = load_preproc_data_german(['age'])
    
elif dataset_used == "compas":
#     dataset_orig = CompasDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 0}]
        unprivileged_groups = [{'sex': 1}]
        dataset_orig = load_preproc_data_compas(['sex'])
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]  
        dataset_orig = load_preproc_data_compas(['race'])

#### Split into train, test and validation

In [3]:
# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

#### Clean up training data and display properties of the data

In [4]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(700, 11)


#### Favorable and unfavorable labels

1.0 2.0


#### Protected attribute names

['sex']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['age', 'sex', 'credit_history=Delay', 'credit_history=None/Paid', 'credit_history=Other', 'savings=500+', 'savings=<500', 'savings=Unknown/None', 'employment=1-4 years', 'employment=4+ years', 'employment=Unemployed']


## Pre-processing: Reweighing

#### Metric for original training data

In [5]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Weights = %f , %f, %f, ..." % (dataset_orig_train.instance_weights[1], dataset_orig_train.instance_weights[2], \
                                    dataset_orig_train.instance_weights[3]))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Weights = 1.000000 , 1.000000, 1.000000, ...
Difference in mean outcomes between unprivileged and privileged groups = -0.100117


#### Reweighing

In [6]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(dataset_orig_train)
dataset_transf_train = RW.transform(dataset_orig_train)

#### Metric for reweighted training data

In [7]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Weights = %8f , %8f, %8f, ..." % (dataset_transf_train.instance_weights[1], dataset_transf_train.instance_weights[2], \
                                    dataset_transf_train.instance_weights[3]))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Weights = 0.822259 , 0.955174, 0.955174, ...
Difference in mean outcomes between unprivileged and privileged groups = 0.000000


## In-processing: Adversarial Debiasing

### Without debiasing

#### Train without debiasing

In [8]:
# Learn parameters with debias set to False
sess = tf.Session() 
plain_model_nodebias = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                           sess=sess)

In [9]:
plain_model_nodebias.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 0.668088
epoch 1; iter: 0; batch classifier loss: 0.631008
epoch 2; iter: 0; batch classifier loss: 0.659046
epoch 3; iter: 0; batch classifier loss: 0.502799
epoch 4; iter: 0; batch classifier loss: 0.608407
epoch 5; iter: 0; batch classifier loss: 0.612942
epoch 6; iter: 0; batch classifier loss: 0.578477
epoch 7; iter: 0; batch classifier loss: 0.527597
epoch 8; iter: 0; batch classifier loss: 0.620062
epoch 9; iter: 0; batch classifier loss: 0.550934
epoch 10; iter: 0; batch classifier loss: 0.584196
epoch 11; iter: 0; batch classifier loss: 0.556542
epoch 12; iter: 0; batch classifier loss: 0.628668
epoch 13; iter: 0; batch classifier loss: 0.492051
epoch 14; iter: 0; batch classifier loss: 0.624512
epoch 15; iter: 0; batch classifier loss: 0.573710
epoch 16; iter: 0; batch classifier loss: 0.559718
epoch 17; iter: 0; batch classifier loss: 0.605144
epoch 18; iter: 0; batch classifier loss: 0.540690
epoch 19; iter: 0; batch classifier loss:

#### Show metrics

In [10]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model_nodebias.predict(dataset_orig_train)
dataset_nodebiasing_valid = plain_model_nodebias.predict(dataset_orig_valid)
dataset_nodebiasing_test = plain_model_nodebias.predict(dataset_orig_test)

In [11]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())


#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.384561
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.348620


In [12]:
# Accuracy
display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())

# Other metrics
metric_test_bef = compute_metrics(dataset_orig_test, dataset_nodebiasing_test, 
                unprivileged_groups, privileged_groups)

#### Plain model - without debiasing - classification metrics

Classification accuracy = 0.673333
Balanced accuracy = 0.5166
Statistical parity difference = -0.3486
Disparate impact = 0.6343
Average odds difference = -0.3912
Equal opportunity difference = -0.2823
Theil index = 0.1596


In [13]:
# Compute scores for ROC
dataset_nodebiasing_valid.scores = plain_model_nodebias.predict_proba(dataset_orig_valid).reshape(-1,1)
dataset_nodebiasing_test.scores = plain_model_nodebias.predict_proba(dataset_orig_test).reshape(-1,1)

### With debiasing

#### Train with debiasing

In [14]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [15]:
# Learn parameters with debias set to True
plain_model_debias = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=True,
                           sess=sess)

In [16]:
plain_model_debias.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 0.734769; batch adversarial loss: 0.652179
epoch 1; iter: 0; batch classifier loss: 0.682632; batch adversarial loss: 0.660819
epoch 2; iter: 0; batch classifier loss: 0.657078; batch adversarial loss: 0.603071
epoch 3; iter: 0; batch classifier loss: 0.590027; batch adversarial loss: 0.633419
epoch 4; iter: 0; batch classifier loss: 0.617300; batch adversarial loss: 0.729237
epoch 5; iter: 0; batch classifier loss: 0.619472; batch adversarial loss: 0.679605
epoch 6; iter: 0; batch classifier loss: 0.596630; batch adversarial loss: 0.744469
epoch 7; iter: 0; batch classifier loss: 0.632444; batch adversarial loss: 0.704037
epoch 8; iter: 0; batch classifier loss: 0.586759; batch adversarial loss: 0.700169
epoch 9; iter: 0; batch classifier loss: 0.616563; batch adversarial loss: 0.586475
epoch 10; iter: 0; batch classifier loss: 0.583529; batch adversarial loss: 0.591621
epoch 11; iter: 0; batch classifier loss: 0.619433; batch adversarial loss:

#### Show metrics

In [17]:
# Apply the plain model to test data
dataset_debiasing_train = plain_model_debias.predict(dataset_orig_train)
dataset_debiasing_test = plain_model_debias.predict(dataset_orig_test)

In [18]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())

#### Plain model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.547368
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.551402


In [19]:
# Accuracy
display(Markdown("#### Plain model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())

#Other metrics
metric_test_bef = compute_metrics(dataset_orig_test, dataset_debiasing_test, 
                unprivileged_groups, privileged_groups)

#### Plain model - with debiasing - classification metrics

Test set: Classification accuracy = 0.600000
Balanced accuracy = 0.5650
Statistical parity difference = 0.5514
Disparate impact = 2.2292
Average odds difference = 0.6141
Equal opportunity difference = 0.4875
Theil index = 0.3484


## Post-processing: Reject Option Classification

#### Show metrics for Test Set

In [20]:
# Metrics for the test set
display(Markdown("#### Test set"))
display(Markdown("##### Raw predictions - No fairness constraints"))
classified_metric_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)

print("Classification accuracy = %f" % classified_metric_test.accuracy())

metric_test_bef = compute_metrics(dataset_orig_test, dataset_nodebiasing_test, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Raw predictions - No fairness constraints

Classification accuracy = 0.673333
Balanced accuracy = 0.5166
Statistical parity difference = -0.3486
Disparate impact = 0.6343
Average odds difference = -0.3912
Equal opportunity difference = -0.2823
Theil index = 0.1596


#### Estimate optimal parameters for the ROC method

In [21]:
# Metric used (should be one of allowed_metrics)
metric_name = "Statistical parity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

In [22]:
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(dataset_orig_valid,dataset_nodebiasing_valid)

In [23]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.5940
Optimal ROC margin = 0.0580


#### Show predictions from Test Set with ROC

In [24]:
# Metrics for the transformed test set
dataset_transf_test = ROC.predict(dataset_nodebiasing_test)

display(Markdown("#### Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
classified_metric_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_transf_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)

print("Classification accuracy = %f" % classified_metric_test.accuracy()) 

metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Transformed predictions - With fairness constraints

Classification accuracy = 0.620000
Balanced accuracy = 0.6472
Statistical parity difference = 0.0396
Disparate impact = 1.0799
Average odds difference = 0.0718
Equal opportunity difference = 0.0457
Theil index = 0.3906


References:

F. Kamiran, and T. Claders,"Data preprocessing techniques for classification without discrimination",
Knowledge and Information Systems, 33(1):1–33, 2012. 

B. H. Zhang, B. Lemoine, and M. Mitchell, "Mitigating UnwantedBiases with Adversarial Learning",
AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.

F. Kamiran, A. Karim, and X. Zhang,  "Decision theory for discrimination-aware classification",
In IEEE International Conference on Data Mining, pp. 924–929, 2012.